Instead of writing separate code for SMA, RSI, and Bollinger, we create a Strategy Agent — a modular class that can run any strategy. This prepares us for orchestrating multiple agents later.

In [1]:
import pandas as pd
import numpy as np

class StrategyAgent:
    def __init__(self, data):
        self.data = data.copy()
        
    def sma_crossover(self, fast=10, slow=50):
        df = self.data.copy()
        df[f"SMA_{fast}"] = df["Close"].rolling(fast).mean()
        df[f"SMA_{slow}"] = df["Close"].rolling(slow).mean()
        df["Signal"] = 0
        df.loc[df[f"SMA_{fast}"] > df[f"SMA_{slow}"], "Signal"] = 1
        df.loc[df[f"SMA_{fast}"] < df[f"SMA_{slow}"], "Signal"] = -1
        df["Position"] = df["Signal"].shift(1).fillna(0)
        df["Return"] = df["Close"].pct_change()
        df["Strategy_Return"] = df["Position"] * df["Return"]
        return df

    def rsi_strategy(self, lower=30, upper=70):
        df = self.data.copy()
        df["Signal"] = 0
        df.loc[df["RSI_14"] < lower, "Signal"] = 1
        df.loc[df["RSI_14"] > upper, "Signal"] = -1
        df["Position"] = df["Signal"].shift(1).fillna(0)
        df["Return"] = df["Close"].pct_change()
        df["Strategy_Return"] = df["Position"] * df["Return"]
        return df

    def bollinger_breakout(self):
        df = self.data.copy()
        df["Signal"] = 0
        df.loc[df["Close"] > df["BB_UPPER"], "Signal"] = 1
        df.loc[df["Close"] < df["BB_LOWER"], "Signal"] = -1
        df["Position"] = df["Signal"].shift(1).fillna(0)
        df["Return"] = df["Close"].pct_change()
        df["Strategy_Return"] = df["Position"] * df["Return"]
        return df


We now test our agent by running SMA, RSI, and Bollinger on Apple’s enriched dataset.

In [2]:
data = pd.read_csv("../data/AAPL_enriched.csv", index_col=0, parse_dates=True)
agent = StrategyAgent(data)

sma_res = agent.sma_crossover()
rsi_res = agent.rsi_strategy()
bb_res = agent.bollinger_breakout()

sma_res.head(), rsi_res.head(), bb_res.head()


(                  Open        High         Low       Close      Volume  \
 Price                                                                    
 2024-10-16  230.527109  231.044690  228.775253  230.706268  34082200.0   
 2024-10-17  232.348639  232.766707  229.452131  231.074570  32993800.0   
 2024-10-18  235.085888  235.085888  232.925942  233.911362  46431500.0   
 2024-10-21  233.363911  235.752803  233.363911  235.384506  36254500.0   
 2024-10-22  232.806535  235.125743  231.522517  234.767410  38846600.0   
 
             SMA_10  SMA_50     SMA_200  Volatility_30     RSI_14      BB_MID  \
 Price                                                                          
 2024-10-16     NaN     NaN  198.110703       0.014182  57.519396  227.021934   
 2024-10-17     NaN     NaN  198.345669       0.014147  58.005551  227.185175   
 2024-10-18     NaN     NaN  198.601710       0.014184  61.646369  227.523599   
 2024-10-21     NaN     NaN  198.876718       0.014199  63.419961  2

We reuse our performance function so every strategy can be compared.

In [3]:
def compute_metrics(df, label="Strategy"):
    strat_curve = (1 + df["Strategy_Return"]).cumprod()
    years = (df.index[-1] - df.index[0]).days / 365.25
    cagr = strat_curve.iloc[-1]**(1/years) - 1
    vol = df["Strategy_Return"].std() * np.sqrt(252)
    sharpe = (df["Strategy_Return"].mean() * 252) / (df["Strategy_Return"].std() * np.sqrt(252))
    roll_max = strat_curve.cummax()
    dd = (strat_curve / roll_max - 1).min()
    return {"Label": label, "CAGR": cagr, "Volatility": vol, "Sharpe": sharpe, "MaxDD": dd}

metrics = [
    compute_metrics(sma_res, "SMA Crossover"),
    compute_metrics(rsi_res, "RSI"),
    compute_metrics(bb_res, "Bollinger")
]

pd.DataFrame(metrics)


,Label,CAGR,Volatility,Sharpe,MaxDD
0,SMA Crossover,-0.121856,0.041243,-3.175536,-0.026330
1,RSI,-0.126123,0.082338,-1.620564,-0.043275
2,Bollinger,0.139849,0.044243,3.024177,-0.007036


We now save our agent as a Python module in /src/ so it can be reused outside Jupyter.

In [4]:
with open("../src/strategy_agent.py", "w") as f:
    f.write("""import pandas as pd
import numpy as np

class StrategyAgent:
    def __init__(self, data):
        self.data = data.copy()
        
    def sma_crossover(self, fast=10, slow=50):
        df = self.data.copy()
        df[f"SMA_{fast}"] = df["Close"].rolling(fast).mean()
        df[f"SMA_{slow}"] = df["Close"].rolling(slow).mean()
        df["Signal"] = 0
        df.loc[df[f"SMA_{fast}"] > df[f"SMA_{slow}"], "Signal"] = 1
        df.loc[df[f"SMA_{fast}"] < df[f"SMA_{slow}"], "Signal"] = -1
        df["Position"] = df["Signal"].shift(1).fillna(0)
        df["Return"] = df["Close"].pct_change()
        df["Strategy_Return"] = df["Position"] * df["Return"]
        return df

    def rsi_strategy(self, lower=30, upper=70):
        df = self.data.copy()
        df["Signal"] = 0
        df.loc[df["RSI_14"] < lower, "Signal"] = 1
        df.loc[df["RSI_14"] > upper, "Signal"] = -1
        df["Position"] = df["Signal"].shift(1).fillna(0)
        df["Return"] = df["Close"].pct_change()
        df["Strategy_Return"] = df["Position"] * df["Return"]
        return df

    def bollinger_breakout(self):
        df = self.data.copy()
        df["Signal"] = 0
        df.loc[df["Close"] > df["BB_UPPER"], "Signal"] = 1
        df.loc[df["Close"] < df["BB_LOWER"], "Signal"] = -1
        df["Position"] = df["Signal"].shift(1).fillna(0)
        df["Return"] = df["Close"].pct_change()
        df["Strategy_Return"] = df["Position"] * df["Return"]
        return df
""")
